In [2]:
%matplotlib inline

In [15]:
from datetime import date
import requests
import json # pretty print only
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.3f}'.format)
import numpy as np

SINGLE_URL_PREFIX = 'https://iss.moex.com/iss/history/engines/futures/markets/forts/securities/'
SINGLE_URL_SUFFIX = '/.jsonp?iss.meta=off&iss.json=extended&'
SINGLE_URL_TIMEFRAME = 'from=2020-05-01&till='

LIST_URL_PREFIX = 'https://iss.moex.com/iss/history/engines/futures/markets/forts/securities/.jsonp?lang=en'

SAVE_PATH = 'moex_securities.f'

def get_time_frame():
    return f'{SINGLE_URL_TIMEFRAME}{date.today()}'

def get_securities_list():
    func_name = 'get_securities_list'
    resp = requests.get(list_url_prefix)
    current_pos, total, page_size = resp.json()['history.cursor']['data'][0]
    pages_to_get = 2 # make it dynamic
    #= total / page_size - 1
    #if total % page_size > 0:
    #    pages_to_get += 1
    print(current_pos, total, page_size)
    counter = 0
    try:
        for row in resp.json()['history']['data']:
            #print(row[2])
            yield row[2] # sec name
            counter += 1
            # debugging
            #if counter > 10:
            #    return
        for num in range(1, pages_to_get+1):
            url = f'{LIST_URL_PREFIX}&start={num * page_size}'
            print(url)
            resp = requests.get(url)
            for row in resp.json()['history']['data']:
                #print(row[2])
                yield row[2] # sec name
                counter += 1        
    except KeyError:
        print(f'{func_name} Unexpected JSON format.')
    print(counter)


def get_security_history(security):
    func_name = 'get_security_history'
    name = security
    print(name)
    time_frame = get_time_frame()
    url = f'{SINGLE_URL_PREFIX}{name}{SINGLE_URL_SUFFIX}{time_frame}'
    #print(url)
    resp = requests.get(url)
    #print(json.dumps(resp.json(), indent=4, sort_keys=True))
    try:
        for row in resp.json()[1]['history']:
            #print(row)
            yield row
    except (KeyError):
        print(f'{func_name} Unexpected JSON format.')

def save_data(df, gentor):
    local_df = pd.DataFrame(gentor)
    #print(local_df.head())
    df.append(local_df)

if __name__ == '__main__':
    """df = ''
    for s in get_securities_list():
        if type(df) == type(''):
            df = pd.DataFrame(get_security_history(s))
        else:
            df = df.append(pd.DataFrame(get_security_history(s)))
    df.reset_index(inplace=True)
    df.to_feather(SAVE_PATH)
    """
    """
    data_frame = pd.read_feather(SAVE_PATH)
    data_frame.fillna(0, inplace=True)
    print(data_frame.groupby('SECID')
          .agg({'VALUE': np.mean, 'VOLUME': np.mean})
          .sort_values(['VALUE', 'VOLUME'], ascending=False))
    """
    


                   VALUE        VOLUME
SECID                                 
SiU0  73,020,490,882.415 1,025,525.491
RIU0  40,753,602,339.050   236,935.849
BRQ0  17,015,833,017.081   561,682.849
GDU0   5,143,408,563.125    40,939.283
EDU0   4,693,625,763.783    58,735.283
EuU0   4,358,586,989.811    54,095.019
SRU0   2,430,144,469.698   115,401.792
MXU0   1,691,071,614.151     6,144.698
GZU0   1,037,380,788.792    55,969.434
SVU0     616,770,770.449    46,395.566
GMU0     406,194,058.038     2,147.340
BRU0     321,732,780.095    10,637.472
LKU0     320,991,695.698     6,300.396
SiZ0     302,072,019.698     4,202.925
MMU0     273,375,706.264     9,937.358
GUU0     267,451,323.169     3,037.830
VBU0      91,448,424.981    25,154.226
SPU0      72,931,474.981     3,761.509
NGN0      63,491,819.971     5,161.264
PTU0      62,475,150.536     1,037.604
RNU0      60,302,399.660     1,629.509
SNU0      55,419,578.453     1,450.604
MNU0      47,576,644.925    11,180.491
RIZ0      47,096,779.130 